In [ ]:
import numpy as np
import pandas as pd
from math import *
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import datasets, model_selection, preprocessing, model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import bernoulli
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
import random
%matplotlib inline 

#####################
# Time Series loading
#####################
df1 = pd.read_csv('/content/AIR.FR-d-20200701.csv', sep=' ', names=['name', 'date', 'time', 'val', 'extra'])
df2 = pd.read_csv('/content/BNP.FR-d-20200701.csv', sep=' ', names=['name', 'date', 'time', 'val', 'extra'])
df3 = pd.read_csv('/content/FP.FR-d-20200701.csv', sep=' ', names=['name', 'date', 'time', 'val', 'extra'])
df4 = pd.read_csv('/content/DG.FR-d-20200701.csv', sep=' ', names=['name', 'date', 'time', 'val', 'extra'])
df5 = pd.read_csv('/content/MC.FR-d-20200701.csv', sep=' ', names=['name', 'date', 'time', 'val', 'extra'])

In [ ]:
#####################
# Data pre-processing
#####################
for c in [df1, df2, df3, df4, df5]:
  c.drop('extra', axis=1, inplace=True)

  c['datetime'] = c[['date', 'time']].agg(' '.join, axis=1)
  c['datetime'] = c['datetime'].astype('datetime64') #'datetime64[ns]'

  c.drop(['date', 'time'], axis=1, inplace=True)

# Time series classes

class TSElement(object):
    def __init__(self,dt, val=0):
        self.dt = dt
        self.val = val
        
    def __str__(self):
        string = []
        
        id_dt= self.dt
        string.append(f'Datetime: {self.dt}')
        
        id_val = self.val
        string.append(f'Value : {self.val}')
        
        return string
        
class TimeSeriesData:
    def __init__(self, df, name, h=3):
        self.name = name
        self.h = h
        self.instances = None
        #self.bs = bs
        
    def __len__(self):
        return len(df) - self.h
    
    def prepare_data(self):
        l = list(zip(df['datetime'],df['val']))
        self.instances  = [[TSElement(*o) for o in l[i:i+self.h]] for i,e in enumerate(l) if (i+self.h) < len(df)]
        print('Size of Dataset:', len(self.instances))

df1 = df1.set_index('datetime')
df2 = df2.set_index('datetime')
df3 = df3.set_index('datetime')
df4 = df4.set_index('datetime')
df5 = df5.set_index('datetime')
##
h = 1000 # We will focus on the first h values of the time series (~250 minutes) to reduce the computation time
df1, df2, df3, df4, df5 = df1[:h], df2[:h], df3[:h], df4[:h], df5[:h]

df = df1.copy() 
df['val1'], df['val2'], df['val3'], df['val4'], df['val5'] = df['val'], df2['val'], df3['val'], d4['val'], df5['val']
df.drop(['val'], axis=1, inplace = True)
df.drop(['name'], axis=1, inplace = True)

# From time series to supervised learning
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations used for prediction
  #
  # n_future ==> no of future observations (predictions) 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [3]:
######################################################################################################
# Causal inference between time series using 
#Owen sampling+Encoder-Decoder multi step LSTM Model With Multivariate Input to compute Shapley values
######################################################################################################

w = 50
Q = 50                     # Integral's discretisation parameter (Rectangle Rule: for simplicity). Other Rules can be used !
M = 50                     # Sample Size for the empirical estimator of the expectation
df2 = df.copy()
d = len(df2.columns)
D = {}                     # Dictionary saving the different causal relations. In fact, each "key" will represente a time serie and D["key"] contains the shapley values related to the causal realtionships going from each time serie to "key" ."                 
for k1 in range(d):
    D['val'+str(k1)] = []
    Sh = np.zeros([1,d])[0]
    for k in range(0,Q+1):
        e = np.zeros([1,d])[0]
        for m in range(M):
            B = bernoulli.rvs(k/Q, size = d)
            while list(B) != list(np.ones([1,d])[0]):
                I = B
                break
            for j in range(d):
                X_j = np.zeros([1,d])[0]
                X_j[j] = 1

                L2, K = [], []
                for i in range(d):
                    L2.append(int(I[i])*df2.columns[i])
                    K.append((int(X_j[i])*df2.columns[i]))
                L1 = list(set(L2 + K))
                L2 = list(set((L2)))
                L1.remove('')
                L2.remove('')

                ###############
                # LSTM forecast
                ###############

                data1 = df2[L1]
                r = int(0.97*len(data1))
                train_df,test_df = data1[:r], data1[r:] 

                # Scaling data
                train = train_df
                scalers={}
                for i in train_df.columns:
                    scaler = MinMaxScaler(feature_range=(-1,1))
                    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
                    s_s = np.reshape(s_s,len(s_s))
                    scalers['scaler_'+ i] = scaler
                    train.replace(np.array(train[i]), s_s)
                test = test_df
                for i in train_df.columns:
                    scaler = scalers['scaler_'+i]
                    s_s = scaler.transform(test[i].values.reshape(-1,1))
                    s_s = np.reshape(s_s,len(s_s))
                    scalers['scaler_'+i] = scaler
                    test.replace(np.array(test[i]), s_s)
                #From time serie to supervised
                n_past = 10
                n_future = 5 
                n_features = len(L1)

                X_train, y_train = split_series(train.values,n_past, n_future)
                X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
                y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
                X_test, y_test = split_series(test.values,n_past, n_future)
                X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
                y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

                # E2D2
                # n_features ==> no of features at each timestep in the data.
                #
                encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
                # encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
                encoder_l1 = tf.compat.v1.keras.layers.CuDNNLSTM(100,return_sequences = True, return_state=True)
                encoder_outputs1 = encoder_l1(encoder_inputs)
                encoder_states1 = encoder_outputs1[1:]
                # encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
                encoder_l2 = tf.compat.v1.keras.layers.CuDNNLSTM(100, return_state=True)
                encoder_outputs2 = encoder_l2(encoder_outputs1[0])
                encoder_states2 = encoder_outputs2[1:]
                #
                decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
                #
                # decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
                decoder_l1 = tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
                # decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
                decoder_l2 = tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
                decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
                #
                model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
                #
                # model_e2d2.summary()

                #Training the model
                reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
                model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
                history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])

                #Predictions
                pred_e2d2=model_e2d2.predict(X_test)

                #Inverse scaling
                for index,i in enumerate(train_df.columns):
                    scaler = scalers['scaler_'+i]
                    pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
                    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
                    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

                #Checking Errors
                b = 0
                for j1 in range(1,6):
                    b += mean_absolute_error(y_test[:,j1-1,k1],pred_e2d2[:,j1-1,k1])
                b = b/n_features

                c1 = 1-b



 
                if L2 == []:
                    c2 = 0
                else:

                    data2 = df2[L2]
                    r = int(0.97*len(data2))
                    train_df,test_df = data2[:r], data2[r:] 

                    # Scaling data
                    train = train_df
                    scalers={}
                    for i in train_df.columns:
                        scaler = MinMaxScaler(feature_range=(-1,1))
                        s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
                        s_s = np.reshape(s_s,len(s_s))
                        scalers['scaler_'+ i] = scaler
                        train.replace(np.array(train[i]), s_s)
                    test = test_df
                    for i in train_df.columns:
                        scaler = scalers['scaler_'+i]
                        s_s = scaler.transform(test[i].values.reshape(-1,1))
                        s_s = np.reshape(s_s,len(s_s))
                        scalers['scaler_'+i] = scaler
                        test.replace(np.array(test[i]), s_s)
                    #From time serie to supervised
                    n_past = 10
                    n_future = 5 
                    n_features = len(L2)

                    X_train, y_train = split_series(train.values,n_past, n_future)
                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
                    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
                    X_test, y_test = split_series(test.values,n_past, n_future)
                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
                    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

                    # E2D2
                    # n_features ==> no of features at each timestep in the data.
                    #
                    encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
                    # encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
                    encoder_l1 = tf.compat.v1.keras.layers.CuDNNLSTM(100,return_sequences = True, return_state=True)
                    encoder_outputs1 = encoder_l1(encoder_inputs)
                    encoder_states1 = encoder_outputs1[1:]
                    # encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
                    encoder_l2 = tf.compat.v1.keras.layers.CuDNNLSTM(100, return_state=True)
                    encoder_outputs2 = encoder_l2(encoder_outputs1[0])
                    encoder_states2 = encoder_outputs2[1:]
                    #
                    decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
                    #
                    # decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
                    decoder_l1 = tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
                    # decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
                    decoder_l2 = tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
                    decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
                    #
                    model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
                    #
                    # model_e2d2.summary()


                    #Training the model 
                    reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
                    model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
                    history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])

                    #Predictions
                    pred_e2d2=model_e2d2.predict(X_test)

                    #Inverse scaling
                    for index,i in enumerate(train_df.columns):
                        scaler = scalers['scaler_'+i]
                        pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
                        y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
                        y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

                    #Checking Errors
                    b = 0
                    for j1 in range(1,6):
                      b += mean_absolute_error(y_test[:,j1-1,k1],pred_e2d2[:,j1-1,k1])
                    b = b/n_features


                    c2 = 1-b
                          ##
                    e[j] += (c1-c2)
        Sh += (e/M)                   
      
    print(k1)
    Sh = (Sh*(1/Q))  # Vector of Features' Shapley Values
    D['val'+str(k1)].append(Sh)